In [1]:
import sys
import torch
sys.path.insert(0, '../../Models')
sys.path.insert(0, '../../Utils')
from _utils import  sample_random_glue_qnli, collect_info_for_metric, save_info, \
                    get_continuation_mapping, get_continuous_attributions, get_continuous_raw_inputs,\
                    attr_normalizing_func
from preload_models import get_qnli_tok_n_model

In [2]:
qnli_data_raw, targets, idxs = sample_random_glue_qnli()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-e6d0a9c9270b6822.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/qnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c46c5e78d2bda808.arrow


In [3]:
tokenizer, model = get_qnli_tok_n_model()

In [4]:
#define some containers to save some info
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import KernelShap
from captum.attr import visualization 

In [6]:
ks = KernelShap(model)

In [7]:
def generate_record(raw_datum, target): #raw_datum expected to be a tuple/list of 2 sentences
    #tokenizer operations
    tokenized = tokenizer(raw_datum, truncation=True, return_offsets_mapping=True)
    offset_mappings = tokenized['offset_mapping']
    #concatenate the two offset_mappings together because they are fed in together
    conti_map = get_continuation_mapping(offset_mappings[0]) + get_continuation_mapping(offset_mappings[1])
    #change the first input_id of the second sentence to be the last input_id of the 1st sentence (i.e. an [END] token))
    tokenized_input_ids = tokenized['input_ids'][0] + \
                        [tokenized['input_ids'][1][i] if i != 0 else tokenized['input_ids'][0][-1] \
                         for i in range(len(tokenized['input_ids'][1]))]
    input_ids = torch.tensor(tokenized_input_ids).unsqueeze(0)
    detokenized = [t.replace('Ġ', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()

     #categorizing results
    pred_class = 'No Entailment' if pred_prob < 0.5 else 'Entailment' 
    true_class = 'No Entailment' if target < 0.5 else 'Entailment' 
    
    #attribution algorithm working
    attribution = ks.attribute(input_emb, n_samples=2000, perturbations_per_eval=200, show_progress=True)
    word_attributions = attribution.squeeze(0).sum(dim=1)
#     word_attributions = attr_normalizing_func(word_attributions)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'No Entailment' if attr_score < 0.5 else 'Entailment'
    convergence_score = None
    
    
#     #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
    print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
# #     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(qnli_data_raw, targets), start=1):
#     example_1 = "How many times has the South Florida/Miami area hosted the Super Bowl?"
#     example_2 = "The South Florida/Miami area has previously hosted the event 10 times (tied for most with New Orleans), with the most recent one being Super Bowl XLIV in 2010."
    
#     example_1 = "When did the third Digimon series begin?"
#     example_2 = "Unlike the two seasons before it and most of the seasons that followed, Digimon Tamers takes a darker and more realistic approach to its story featuring Digimon who do not reincarnate after their deaths and more complex character development in the original Japanese."
#     datum_raw, target = [example_1, example_2], 1
    print(f'Raw datum: {datum_raw}') #datum expected to be a list of 2 sentences
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record])) 


/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/lime.py:1088: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


Raw datum: ['What would a teacher assess the levels of a student on?', 'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.']
GT target: 0


Kernel Shap attribution: 100%|██████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.79s/it]


word attr tensor([-0.1470,  0.0384,  0.0819,  0.0463,  0.0348, -0.0333,  0.0771,  0.0656,
        -0.0253, -0.1230,  0.0357, -0.0458,  0.1313,  0.1006, -0.0847,  0.0877,
        -0.2107, -0.1592,  0.0527, -0.1756, -0.0605, -0.0348, -0.0767,  0.0810,
        -0.4181, -0.2079, -0.2094,  0.1392, -0.0425, -0.2764,  0.0297, -0.2987,
         0.0870,  0.0601, -0.1505, -0.1758,  0.0783, -0.1093, -0.0818,  0.0589,
         0.0180,  0.0996, -0.0075, -0.0228, -0.1811,  0.0799,  0.0282,  0.0939,
        -0.0946,  0.1453, -0.2430, -0.1492,  0.0476, -0.1073, -0.0589, -0.0647,
         0.1685, -0.0916,  0.0414])
conti attr [tensor(-0.1470), tensor(0.0384), tensor(0.0819), tensor(0.0463), tensor(0.0348), tensor(-0.0333), tensor(0.0771), tensor(0.0656), tensor(-0.0253), tensor(-0.1230), tensor(0.0357), tensor(0.0855), tensor(0.1006), tensor(-0.0847), tensor(0.0877), tensor(-0.3699), tensor(0.0527), tensor(-0.1756), tensor(-0.0605), tensor(-0.0348), tensor(-0.0767), tensor(0.0810), tensor(-0.4181), ten

/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/lime.py:1088: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


...

/home/user/anaconda3/envs/py38/lib/python3.9/site-packages/captum/attr/_core/lime.py:1088: UserWarning: Attempting to construct interpretable model with > 10000 features.This can be very slow or lead to OOM issues. Please provide a featuremask which groups input features to reduce the number of interpretablefeatures. 
  warnings.warn(


<IPython.core.display.HTML object>
Raw datum: ['If the apparant force of two fermions is attractive, what is the spin function?', 'If two identical fermions (e.g. electrons) have a symmetric spin function (e.g. parallel spins) the spatial variables must be antisymmetric (i.e. they exclude each other from their places much as if there was a repulsive force), and vice versa, i.e. for antiparallel spins the position variables must be symmetric (i.e. the apparent force must be attractive).']
GT target: 1


Kernel Shap attribution: 100%|██████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.30s/it]


word attr tensor([ 0.0170,  0.0240,  0.0824,  0.0322, -0.0189,  0.0423,  0.0585,  0.0297,
         0.0474, -0.1008, -0.0174,  0.0050, -0.0551, -0.0171,  0.0103,  0.0304,
         0.0408,  0.0515,  0.0200,  0.0088,  0.0573, -0.7455, -0.0284,  0.0858,
         0.0246,  0.0754, -0.0601,  0.0249,  0.0688, -0.1444,  0.0326, -0.0167,
        -0.0289,  0.0081,  0.0481, -0.1436,  0.0175, -0.0033,  0.0434, -0.0088,
         0.0762,  0.0345, -0.2878, -0.0484, -0.0623, -0.0058,  0.0492, -0.1245,
         0.0057, -0.0278, -0.0205,  0.0680,  0.0129,  0.0382,  0.0787,  0.0642,
        -0.1384,  0.0501,  0.0355,  0.0485, -0.1343,  0.0258,  0.0494,  0.0419,
         0.0193, -0.0297,  0.0397,  0.0532,  0.0342, -0.0255, -0.0298,  0.0466,
        -0.0422,  0.0295,  0.0388,  0.0104, -0.0703,  0.0163,  0.0239,  0.0030,
         0.0307, -0.0888,  0.0293, -0.0243, -0.0787,  0.0597, -0.0789,  0.0058,
         0.0480,  0.0279,  0.0456,  0.0304,  0.0173,  0.0406,  0.0378,  0.0365,
         0.0563,  0.0876, -0.0

<IPython.core.display.HTML object>


In [9]:
save_info(idxs, qnli_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, fname='kernel_shap_out.pkl')

{'indices': [4013,
  2281,
  2771,
  5109,
  1852,
  2016,
  642,
  3912,
  4427,
  2330,
  4581,
  2611,
  5186,
  2700,
  4920,
  4335,
  1246,
  2811,
  5290,
  5385,
  4774,
  653,
  2816,
  916,
  3297,
  3459,
  1475,
  1287,
  4548,
  1476,
  2328,
  2208,
  2772,
  5301,
  869,
  4968,
  4456,
  4711,
  1929,
  634,
  3774,
  5389,
  497,
  4901,
  4602,
  831,
  1292,
  1087,
  3159,
  3492],
 'raw_data': [['What would a teacher assess the levels of a student on?',
   'For example, an experienced teacher and parent described the place of a teacher in learning as follows: "The real bulk of learning takes place in self-study and problem solving with a lot of feedback around that loop.'],
  ['What company created Doctor Who?',
   'Who character by BBC Television in the early 1960s, a myriad of stories have been published about Doctor Who, in different media: apart from the actual television episodes that continue to be produced by the BBC, there have also been novels, comics, sho